In [65]:
import json
import pandas as pd
import altair as alt
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
# alt.data_transformers.enable("vegafusion")
# Solve a javascript error by explicitly setting the renderer
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

In [66]:
events = {}
nations = ['Italy', 'England', 'Germany', 'France', 'Spain', 'European_Championship', 'World_Cup']
for nation in nations:
    with open('./data/events/events_%s.json' % nation) as json_data:
        events[nation] = json.load(json_data)


In [8]:
# Replicating Static Graph
def in_match_evolution(event_name='Goal', event_tag=101):
    event_time = []
    excluded_tags = ['e1', 'e2', 'p']
    for nation in nations:
        for ev in events[nation]:
            if all(tag['id'] not in excluded_tags for tag in ev['tags']) and any(tag['id'] == event_tag for tag in ev['tags']):
                event_time.append([ev['matchId'], ev['matchPeriod'], ev['eventSec']])

    df_event = pd.DataFrame(event_time, columns=['matchId', 'matchPeriod', 'eventSec'])

    df_event['eventSec'] = df_event['eventSec'] / 60  # Convert eventSec to minutes

    # Create bins
    bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, np.inf]
    labels = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '40-45', '>45',
              '45-50', '50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '>90']
    df_event['binned'] = pd.cut(df_event['eventSec'], bins=bins, labels=False)
    df_event['binned'] = df_event['binned'].apply(lambda x: labels[x])

    # Exclude specific bins
    df_event = df_event[~df_event['binned'].isin(['45-50', '5-10'])]

    # Count events in each bin for only 1H and 2H
    df_counts = df_event[df_event['matchPeriod'].isin(['1H', '2H'])].groupby(['matchPeriod', 'binned']).size().reset_index(name='count')

    # Create Altair Chart
    chart = alt.Chart(df_counts).mark_bar().encode(
        x=alt.X('binned:N', title='Match Time (min)', axis=alt.Axis(labels=True, labelAngle=90)),
        y=alt.Y('count:Q', title='%s (n)' % event_name),
        color=alt.Color('matchPeriod:N', title='Half', scale=alt.Scale(range=['#1f77b4', '#2ca02c'])),
        column=alt.Column('matchPeriod:N', title='Half', header=alt.Header(labelOrient='top'))
    ).properties(
        width=150,  # Adjust width here
        height=400
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=15
    ).configure_header(
        labelFontSize=15
    )

    return chart


In [81]:
goals = in_match_evolution()
goals.configure_legend(
    titleFontSize=15,
    labelFontSize=12
).configure_title(
    fontSize=20
)


/Users/aloedeguia/opt/anaconda3/envs/si649/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [39]:
yellowcard = in_match_evolution(event_name = 'Yellow Card', event_tag = 1702)
yellowcard.configure_legend(
    titleFontSize=15,
    labelFontSize=12
).configure_title(
    fontSize=20
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [40]:
redcard = in_match_evolution(event_name = 'Red Card', event_tag = 1701)
redcard.configure_legend(
    titleFontSize=15,
    labelFontSize=12
).configure_title(
    fontSize=20
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [67]:
# Create a dropdown for nation selection
select_nation=alt.selection_point(
    fields=['Nation'],
    value=nations[0],
    bind=alt.binding_select(options=nations),
    on="keyup", #disable
    clear="false")


In [78]:
select_nation=alt.selection_point(
    fields=['Nation'],
    value=nations[0],
    bind=alt.binding_select(options=nations),
    on="keyup", #disable
    clear="false")

def in_match_evolution(event_name='Goal', event_tag=101):
    event_time = []
    excluded_tags = ['e1', 'e2', 'p']
    for nation in nations:
        for ev in events[nation]:
            if all(tag['id'] not in excluded_tags for tag in ev['tags']) and any(tag['id'] == event_tag for tag in ev['tags']):
                event_time.append([ev['matchId'], ev['matchPeriod'], ev['eventSec'], nation])

    df_event = pd.DataFrame(event_time, columns=['matchId', 'matchPeriod', 'eventSec', 'Nation'])

    df_event['eventSec'] = df_event['eventSec'] / 60  # Convert eventSec to minutes

    # Create bins
    bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, np.inf]
    labels = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '40-45', '>45',
              '45-50', '50-55', '55-60', '60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '>90']
    df_event['binned'] = pd.cut(df_event['eventSec'], bins=bins, labels=False)
    df_event['binned'] = df_event['binned'].apply(lambda x: labels[x])

    # Exclude specific bins
    df_event = df_event[~df_event['binned'].isin(['45-50', '5-10'])]

    # Count events in each bin for only 1H and 2H
    df_counts = df_event[df_event['matchPeriod'].isin(['1H', '2H'])].groupby(['matchPeriod', 'binned', 'Nation']).size().reset_index(name='count')
    # df_counts.sample(10)
    # Create Altair Chart
    chart = alt.Chart(df_counts).mark_bar().encode(
        x=alt.X('binned:N', title='Match Time (min)', axis=alt.Axis(labels=True, labelAngle=90)),
        y=alt.Y('count:Q', title='%s (n)' % event_name).scale(domain=(0, 140)),
        color=alt.Color('matchPeriod:N', title='Half', scale=alt.Scale(range=['#1f77b4', '#2ca02c'])),
        column=alt.Column('matchPeriod:N', title='Half', header=alt.Header(labelOrient='top')),
        opacity=alt.condition(select_nation, alt.value(1), alt.value(0.2))
    ).properties(
        width=150,  # Adjust width here
        height=400
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=15
    ).configure_header(
        labelFontSize=15
    ).add_selection(
        select_nation
    ).transform_filter(
        select_nation)
    
    return chart


In [82]:
goals = in_match_evolution()
goals.configure_legend(
    titleFontSize=15,
    labelFontSize=12
).configure_title(
    fontSize=20
)
goals.save('goals.html')


/Users/aloedeguia/opt/anaconda3/envs/si649/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


ValueError: Objects with "config" attribute cannot be used within HConcatChart. Consider defining the config attribute in the HConcatChart object instead.

In [ ]:
goals.save('goals.html')
